In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

from concepts import dynamic_concepts, linear_regression
import env
from policy import ConvNet

model_name = "net"
session_name = "endeavor"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = dynamic_concepts.game_start_sequence

CONCEPT_NAME = dynamic_concepts.game_start_sequence.__name__

CASES_TO_SAMPLE = 2500 # 25000

In [33]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [37]:
positive_cases = []
negative_cases = []

pos, neg = CONCEPT_FUNC()

positive_cases.extend(pos)
negative_cases.extend(neg)

print(f"Positive cases: {len(positive_cases)}")
print(f"Negative cases: {len(negative_cases)}")

INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpbzslwdkl/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpbzslwdkl/assets
2024-02-02 15:54:42.072232: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpbzslwdkl
2024-02-02 15:54:42.074001: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2024-02-02 15:54:42.074013: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpbzslwdkl
2024-02-02 15:54:42.078619: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-02 15:54:42.118472: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmpbzslwdkl
2024-02-02 15:54:42.134186: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 61954 microseconds.


0
143
1
4
20
22
0
1
4
17
19
0
6
Positive cases: 5
Negative cases: 5
